In [ ]:
# Check core SDK version number
import azureml.core
import os

print("SDK version:", azureml.core.VERSION)

In [ ]:
2. 모델 등록

model_version = "03_051_01"
file_name = '{}.pickle'.format(model_version)
print(file_name)

In [ ]:
from azureml.core.model import Model

#library_version = "DL"+sklearn.__version__.replace(".","x")
model = Model.register(model_path = "data/pkl/{}".format(file_name),
                       model_name = model_version,
                       tags = {'area': "IoT Edge", 'type': "azureml-automl"},
                       description = "ML=03_051_01, Sys=04.01.00",
                       workspace = ws)

In [ ]:
from azureml.core.model import Model
model = Model.get_model_path(model_version, _workspace=ws)
model

In [ ]:
pip_packages = ["azureml-defaults",
                "azureml==0.2.7",
                "azureml-core==1.24.0",
                "azureml-sdk==1.24.0",
                "azureml-automl-core==1.24.0",
                "azureml-automl-runtime==1.24.0",
                "azure-storage-blob==12.6.0",
                "numpy==1.18.5",
                "pandas==0.25.3",
                "sqlalchemy==1.3.21",
                "h5py==2.10.0",
                "tqdm==4.54.0",
                "obspy==1.2.2",
                "mysql-connector-python==8.0.18",
                "joblib==0.14.1",
                "packaging==20.7",
                "xgboost==0.90",
                "noisereduce==1.1.0"]

lib_config_load = ['pyyaml']
lib_clfs = ["gxx_linux-64",
            "gcc_linux-64",
            "librosa==0.8.0"]
#lib_clfs = ['tensorflow==1.1']
conda_packages = lib_clfs + lib_config_load

print('pip packages:', pip_packages)
print('conda packasges:', conda_packages)

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment

conda_deps  = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
conda_deps.add_channel('conda-forge')
conda_deps.add_channel('defaults')

myenv = Environment(name='myenv')
myenv.python.conda_dependencies = conda_deps

# myenv.docker.base_image = 'mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04'


#myenv.docker.base_image = 'mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04'
# myenv.docker.base_image = 'mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu18.04'


# with open("data/myenv.yml","w") as f:
#     f.write(myenv.serialize_to_string())

In [ ]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score_iot.py", environment=myenv, source_directory="../src")
package_service = Model.package(workspace=ws,
                                models=[model],
                                image_name="mc-mlmodule",
                                image_label='30',
                                inference_config=inference_config)

In [ ]:
package_service.wait_for_creation(show_output=True)